In [25]:
from sklearn.model_selection import GroupKFold
# from sklearn.linear_model import SGDClassifier # Not used in the final classical list
# from sklearn.neural_network import MLPClassifier # Not used
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis # Not used
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from scipy import signal
from scipy.signal import spectrogram, welch

import matplotlib.pyplot as plt
import scipy.io as sio
import neurokit2 as nk
import seaborn as sns

import pandas as pd
import numpy as np
import time
import gc # For garbage collection

# Deep Learning and SOTA Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, BatchNormalization, ReLU, MaxPooling2D, AveragePooling2D,
                                     Flatten, Dense, Dropout, Input, Add, Activation, DepthwiseConv2D,
                                     SpatialDropout2D, ELU, SeparableConv2D, GlobalAveragePooling2D, Resizing)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda

# KerasTuner for Hyperparameter Optimization
# Install with: pip install keras-tuner
import keras_tuner as kt

# Pre-trained model
from tensorflow.keras.applications import EfficientNetB0

In [8]:
# Define path to the dataset
path = "C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\DREAMER.mat" # Ensure this path is correct
raw = sio.loadmat(path)

# Parameters for CNN Spectrograms
NPERSEG_CNN = 64  # nperseg for spectrogram generation
NOVERLAP_CNN = NPERSEG_CNN // 2 # noverlap for spectrogram generation

# Parameters for EEGNet
EEGNET_SAMPLING_RATE = 128 # Hz, specific to DREAMER EEG
EEGNET_CHANNELS = 14
EEGNET_SAMPLES_PER_SUBJECT = 128 # Corresponds to 1 second of EEG data for EEGNet input

# EfficientNetB0 parameters
EFFNET_IMG_SIZE = 32 # Minimum typical size for EfficientNet, resize spectrograms to this
EFFNET_CHANNELS = 3 # EfficientNet expects 3 channels


In [9]:
def feat_extract_ECG(raw_data, model_type='RNN', sampling_rate=256, fixed_length=None,
                     nperseg_spec=None, noverlap_spec=None):
    extracted_data_ts = []
    extracted_data_spec = []
    n_participants = 23
    n_videos = 18

    for participant in range(n_participants):
        for video in range(n_videos):
            basl_left = raw_data["DREAMER"][0, 0]["Data"][0, participant]["ECG"][0, 0]\
                                ["baseline"][0, 0][video, 0][:, 0]
            stim_left = raw_data["DREAMER"][0, 0]["Data"][0, participant]["ECG"][0, 0]\
                                ["stimuli"][0, 0][video, 0][:, 0]
            basl_right = raw_data["DREAMER"][0, 0]["Data"][0, participant]["ECG"][0, 0]\
                                 ["baseline"][0, 0][video, 0][:, 1]
            stim_right = raw_data["DREAMER"][0, 0]["Data"][0, participant]["ECG"][0, 0]\
                                 ["stimuli"][0, 0][video, 0][:, 1]

            try:
                signals_b_l, _ = nk.ecg_process(basl_left, sampling_rate=sampling_rate)
                signals_s_l, _ = nk.ecg_process(stim_left, sampling_rate=sampling_rate)
                signals_b_r, _ = nk.ecg_process(basl_right, sampling_rate=sampling_rate)
                signals_s_r, _ = nk.ecg_process(stim_right, sampling_rate=sampling_rate)

                ecg_clean_left = signals_s_l["ECG_Clean"].values - np.mean(signals_b_l["ECG_Clean"].values)
                ecg_clean_right = signals_s_r["ECG_Clean"].values - np.mean(signals_b_r["ECG_Clean"].values)
            except Exception as e:
                # print(f"Neurokit processing error for P{participant+1} V{video+1} ECG: {e}. Using zeros.")
                ecg_clean_left = np.zeros(fixed_length if fixed_length else sampling_rate) # fallback
                ecg_clean_right = np.zeros(fixed_length if fixed_length else sampling_rate)


            sample_time_series = np.stack([ecg_clean_left, ecg_clean_right], axis=-1)

            if fixed_length is not None:
                T = sample_time_series.shape[0]
                if T > fixed_length:
                    sample_time_series = sample_time_series[:fixed_length, :]
                elif T < fixed_length:
                    pad_width = fixed_length - T
                    sample_time_series = np.pad(sample_time_series, ((0, pad_width), (0, 0)), mode='constant')

            if model_type.upper() == 'RNN':
                extracted_data_ts.append(sample_time_series.flatten())
            elif model_type.upper() == 'CNN':
                spec_channels = []
                for ch_idx in range(sample_time_series.shape[1]):
                    signal_ch = sample_time_series[:, ch_idx]
                    if np.std(signal_ch) < 1e-9:
                        n_freq_bins = nperseg_spec // 2 + 1
                        n_time_bins = (fixed_length - noverlap_spec) // (nperseg_spec - noverlap_spec)
                        Sxx = np.zeros((n_freq_bins, n_time_bins))
                    else:
                        _, _, Sxx = spectrogram(signal_ch, fs=sampling_rate, nperseg=nperseg_spec, noverlap=noverlap_spec)
                    spec_channels.append(np.log1p(Sxx))
                sample_spec = np.stack(spec_channels, axis=-1)
                extracted_data_spec.append(sample_spec)

    if model_type.upper() == 'RNN':
        return pd.DataFrame(extracted_data_ts)
    elif model_type.upper() == 'CNN':
        return np.array(extracted_data_spec)
    else:
        raise ValueError("model_type must be either 'RNN' or 'CNN'")

def feat_extract_EEG(raw_data, model_type='RNN_FLAT', sampling_rate=128, fixed_length=None, # EEG sampling rate is 128Hz in DREAMER
                     nperseg_spec=None, noverlap_spec=None):
    """
    Extract EEG data.
    model_type:
      'RNN_FLAT': Flattens time series for classical ML. (fixed_length = EEGNET_SAMPLES_PER_SUBJECT)
      'CNN_SPECTROGRAM': Spectrograms for CNNs. (fixed_length = fixed_length_cnn)
      'EEGNET_INPUT': Shaped for EEGNet (N, C, T, 1) or (N, T, C) then reshaped. (fixed_length = EEGNET_SAMPLES_PER_SUBJECT)
    """
    extracted_data_rnn_flat = []
    extracted_data_cnn_spec = []
    extracted_data_eegnet = [] # List of (fixed_length, n_channels)

    n_participants = 23
    n_videos = 18
    n_channels = 14

    for participant in range(n_participants):
        for video in range(n_videos):
            channels_data_single_trial = [] # For one trial, (fixed_length, n_channels)
            for i in range(n_channels):
                basl = raw_data["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]\
                                ["baseline"][0, 0][video, 0][:, i]
                stim = raw_data["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]\
                                ["stimuli"][0, 0][video, 0][:, i]
                corrected_signal = stim - np.mean(basl) # Simple baseline correction
                channels_data_single_trial.append(corrected_signal)

            # Before padding, stack to (timesteps_original, n_channels)
            sample_time_series_unpadded = np.stack(channels_data_single_trial, axis=-1)

            # Pad/truncate to fixed_length
            current_fixed_length = fixed_length # Use the passed fixed_length
            T_orig = sample_time_series_unpadded.shape[0]
            
            if T_orig > current_fixed_length:
                sample_time_series = sample_time_series_unpadded[:current_fixed_length, :]
            elif T_orig < current_fixed_length:
                pad_width_time = current_fixed_length - T_orig
                sample_time_series = np.pad(sample_time_series_unpadded, ((0, pad_width_time), (0, 0)), mode='constant')
            else:
                sample_time_series = sample_time_series_unpadded # No padding/truncation needed

            # --- Process based on model_type ---
            if model_type.upper() == 'RNN_FLAT':
                extracted_data_rnn_flat.append(sample_time_series.flatten())

            elif model_type.upper() == 'CNN_SPECTROGRAM':
                spec_channels = []
                for ch_idx in range(sample_time_series.shape[1]): # Iterate over 14 channels
                    signal_ch = sample_time_series[:, ch_idx]
                    if np.std(signal_ch) < 1e-9:
                        n_freq_bins = nperseg_spec // 2 + 1
                        n_time_bins = (current_fixed_length - noverlap_spec) // (nperseg_spec - noverlap_spec)
                        Sxx = np.zeros((n_freq_bins, n_time_bins))
                    else:
                        _, _, Sxx = spectrogram(signal_ch, fs=sampling_rate, nperseg=nperseg_spec, noverlap=noverlap_spec)
                    spec_channels.append(np.log1p(Sxx))
                sample_spec = np.stack(spec_channels, axis=-1) # (n_freq, n_time, 14)
                extracted_data_cnn_spec.append(sample_spec)

            elif model_type.upper() == 'EEGNET_INPUT':
                # Expected shape (fixed_length, n_channels) for now, will be reshaped later
                extracted_data_eegnet.append(sample_time_series)


    if model_type.upper() == 'RNN_FLAT':
        return pd.DataFrame(extracted_data_rnn_flat)
    elif model_type.upper() == 'CNN_SPECTROGRAM':
        return np.array(extracted_data_cnn_spec) # (n_samples, n_freq, n_time, 14)
    elif model_type.upper() == 'EEGNET_INPUT':
        # Return as (n_samples, fixed_length, n_channels)
        # Reshape to (n_samples, n_channels, fixed_length, 1) or (n_samples, 1, n_channels, fixed_length) later
        return np.array(extracted_data_eegnet)
    else:
        raise ValueError("model_type must be 'RNN_FLAT', 'CNN_SPECTROGRAM', or 'EEGNET_INPUT'")


def participant_affective(raw_data):
    # (Identical to your original function)
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw_data["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            a[participant, video, 1] = (raw_data["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            a[participant, video, 2] = int(participant+1)
            a[participant, video, 3] = int(video+1)
            a[participant, video, 4] = ["Searching for Bobby Fischer", "D.O.A.", "The Hangover", "The Ring", "300", "National Lampoon\'s VanWilder", "Wall-E", "Crash", "My Girl", "The Fly", "Pride and Prejudice", "Modern Times", "Remember the Titans", "Gentlemans Agreement", "Psycho", "The Bourne Identitiy", "The Shawshank Redemption", "The Departed"][video]
            a[participant, video, 5] = ["calmness", "surprise", "amusement", "fear", "excitement", "disgust", "happiness", "anger", "sadness", "disgust", "calmness", "amusement", "happiness", "anger", "fear", "excitement", "sadness", "surprise"][video]
            a[participant, video, 6] = int(raw_data["DREAMER"][0, 0]["Data"] [0, participant]["ScoreValence"] [0, 0][video, 0])
            a[participant, video, 7] = int(raw_data["DREAMER"][0, 0]["Data"] [0, participant]["ScoreArousal"] [0, 0][video, 0])
            a[participant, video, 8] = int(raw_data["DREAMER"][0, 0]["Data"] [0, participant]["ScoreDominance"] [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])), columns=["age", "gender", "participant", "video", "video_name", "target_emotion", "valence", "arousal", "dominance"])
    for col in ["age", "participant", "video", "valence", "arousal", "dominance"]: b[col] = b[col].astype(int)
    b["gender"] = b["gender"].astype(str)
    return b


## Data Augmentation: SpecAugment

In [10]:
class SpecAugment(tf.keras.layers.Layer):
    """Spectrogram augmentation layer performing frequency and time masking.
       Assumes channels_last format (batch, freq, time, channels).
    """
    def __init__(self, freq_mask_param, time_mask_param, num_freq_masks, num_time_masks, name="spec_augment", **kwargs):
        super(SpecAugment, self).__init__(name=name, **kwargs)
        self.freq_mask_param = freq_mask_param
        self.time_mask_param = time_mask_param
        self.num_freq_masks = num_freq_masks
        self.num_time_masks = num_time_masks

    def call(self, inputs, training=None):
        if not training:
            return inputs

        augmented_inputs = inputs
        input_shape = tf.shape(inputs)
        batch_size, num_freq_bins, num_time_frames, num_channels = input_shape[0], input_shape[1], input_shape[2], input_shape[3]
        num_freq_bins_float = tf.cast(num_freq_bins, tf.float32)
        num_time_frames_float = tf.cast(num_time_frames, tf.float32)


        for _ in range(self.num_freq_masks):
            f = tf.random.uniform([], minval=0, maxval=self.freq_mask_param, dtype=tf.int32)
            f0 = tf.random.uniform([], minval=0, maxval=num_freq_bins - f, dtype=tf.int32)
            mask = tf.concat([tf.ones((batch_size, f0, num_time_frames, num_channels)),
                              tf.zeros((batch_size, f, num_time_frames, num_channels)),
                              tf.ones((batch_size, num_freq_bins - f0 - f, num_time_frames, num_channels))], axis=1)
            augmented_inputs = augmented_inputs * mask
        
        for _ in range(self.num_time_masks):
            t = tf.random.uniform([], minval=0, maxval=self.time_mask_param, dtype=tf.int32)
            t0 = tf.random.uniform([], minval=0, maxval=num_time_frames - t, dtype=tf.int32)
            mask = tf.concat([tf.ones((batch_size, num_freq_bins, t0, num_channels)),
                              tf.zeros((batch_size, num_freq_bins, t, num_channels)),
                              tf.ones((batch_size, num_freq_bins, num_time_frames - t0 - t, num_channels))], axis=2)
            augmented_inputs = augmented_inputs * mask
            
        return augmented_inputs

    def get_config(self):
        config = super().get_config()
        config.update({
            "freq_mask_param": self.freq_mask_param,
            "time_mask_param": self.time_mask_param,
            "num_freq_masks": self.num_freq_masks,
            "num_time_masks": self.num_time_masks,
        })
        return config

## Part 1: Classical ML Classifiers (using RNN-style flattened features)

In [11]:
fixed_length_eeg_rnn = EEGNET_SAMPLES_PER_SUBJECT # 128 samples for EEG (1s)
fixed_length_ecg_rnn = 256  # 1 second of ECG data at 256 Hz for consistency with previous setup

print("Extracting RNN-style features for classical ML...")
df_EEG_rnn = feat_extract_EEG(raw, model_type='RNN_FLAT', sampling_rate=EEGNET_SAMPLING_RATE, fixed_length=fixed_length_eeg_rnn)
df_ECG_rnn = feat_extract_ECG(raw, model_type='RNN', sampling_rate=256, fixed_length=fixed_length_ecg_rnn) # Using original ECG feat extract
df_features_rnn = pd.concat([df_EEG_rnn, df_ECG_rnn], axis=1)
df_features_rnn.columns = [f"feat_{i}" for i in range(df_features_rnn.shape[1])]

df_participant_affective = participant_affective(raw)
df_rnn = pd.concat([df_features_rnn, df_participant_affective.reset_index(drop=True)], axis=1) # reset_index if needed

data_rnn = df_rnn.loc[(df_rnn['target_emotion'] == 'anger') |
                      (df_rnn['target_emotion'] == 'fear') |
                      (df_rnn['target_emotion'] == 'calmness')].copy()

data_rnn['stress_bin'] = data_rnn['target_emotion'].map({'anger': 1, 'fear': 1, 'calmness': 0})

X_rnn = np.array(data_rnn.iloc[:, 0:df_features_rnn.shape[1]])
y_rnn = np.array(data_rnn['stress_bin'])
groups_rnn = np.array(data_rnn['participant'])

print(f"Shape of X_rnn for classical ML: {X_rnn.shape}")

def run_clf_cv(clf, X, y, groups, n_splits=10):
    cv = GroupKFold(n_splits=n_splits)
    scores_list, runtimes_list = [], []
    if np.isnan(X).any(): print("Warning: NaNs in X for classical. Imputer should handle.")
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y, groups)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        clf.fit(X_train, y_train)
        start_time = time.time()
        score = clf.score(X_test, y_test)
        runtime = time.time() - start_time
        scores_list.append(score); runtimes_list.append(runtime)
    return scores_list, runtimes_list

results_classical = []
names_classical = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process", 
                   "Decision Tree", "Random Forest", "AdaBoost", "Naive Bayes"]
classifiers_classical = [
    KNeighborsClassifier(3), SVC(kernel="linear", C=0.025, probability=True), SVC(gamma=2, C=1, probability=True),
    GaussianProcessClassifier(1.0 * RBF(1.0)), DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=100, max_features='sqrt'), AdaBoostClassifier(), GaussianNB()]

print("\nRunning classical ML models...")
for name, classifier_model in zip(names_classical, classifiers_classical):
    pipeline = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(), classifier_model)
    print(f"Training {name}...")
    try:
        # Reduce n_splits for faster initial testing if needed
        scores, runtimes = run_clf_cv(pipeline, X_rnn, y_rnn, groups_rnn, n_splits=5) 
        results_classical.append([name, np.mean(scores), np.std(scores), np.mean(runtimes)])
        print(f"{name}: Mean Acc = {np.mean(scores):.4f} +/- {np.std(scores):.4f}")
    except Exception as e:
        print(f"Could not run {name}. Error: {e}")
        results_classical.append([name, np.nan, np.nan, np.nan])
results_classical_df = pd.DataFrame(results_classical, columns=['name', 'mean_score', 'std_score', 'mean_runtime'])
print("\nClassical ML Results:\n", results_classical_df)



Extracting RNN-style features for classical ML...
Shape of X_rnn for classical ML: (138, 2304)

Running classical ML models...
Training Nearest Neighbors...
Nearest Neighbors: Mean Acc = 0.5950 +/- 0.0759
Training Linear SVM...
Linear SVM: Mean Acc = 0.6517 +/- 0.0186
Training RBF SVM...
RBF SVM: Mean Acc = 0.6667 +/- 0.0000
Training Gaussian Process...
Gaussian Process: Mean Acc = 0.6600 +/- 0.0133
Training Decision Tree...
Decision Tree: Mean Acc = 0.5383 +/- 0.0869
Training Random Forest...
Random Forest: Mean Acc = 0.6217 +/- 0.0557
Training AdaBoost...
AdaBoost: Mean Acc = 0.5583 +/- 0.0756
Training Naive Bayes...
Naive Bayes: Mean Acc = 0.4833 +/- 0.1109

Classical ML Results:
                 name  mean_score  std_score  mean_runtime
0  Nearest Neighbors    0.595000   0.075939      0.069536
1         Linear SVM    0.651667   0.018559      0.004673
2            RBF SVM    0.666667   0.000000      0.003163
3   Gaussian Process    0.660000   0.013333      0.003397
4      Decision T

## Part 2: CNN / Advanced DL Classifiers (Spectrogram & Time-Series Features)

In [12]:
# --- Data Prep for CNNs (Spectrograms) ---
fixed_length_cnn = 256 # Num samples for spectrogram input signal (2s for EEG@128Hz, 1s for ECG@256Hz)
print("\nExtracting CNN-style spectrogram features...")
data_EEG_cnn_spec = feat_extract_EEG(raw, model_type='CNN_SPECTROGRAM', sampling_rate=EEGNET_SAMPLING_RATE, fixed_length=fixed_length_cnn,
                                nperseg_spec=NPERSEG_CNN, noverlap_spec=NOVERLAP_CNN)
data_ECG_cnn_spec = feat_extract_ECG(raw, model_type='CNN', sampling_rate=256, fixed_length=fixed_length_cnn,
                                nperseg_spec=NPERSEG_CNN, noverlap_spec=NOVERLAP_CNN)

X_cnn_spectrograms = np.concatenate((data_EEG_cnn_spec, data_ECG_cnn_spec), axis=-1)
print(f"Shape of combined X_cnn_spectrograms: {X_cnn_spectrograms.shape}") # (414, 33, 7, 16)

# Align with targets (already done for data_rnn, can reuse indices)
df_cnn_targets = df_participant_affective.copy()
data_cnn_filtered_df = df_cnn_targets.loc[(df_cnn_targets['target_emotion'] == 'anger') |
                                          (df_cnn_targets['target_emotion'] == 'fear') |
                                          (df_cnn_targets['target_emotion'] == 'calmness')].copy()
idx_filter = data_cnn_filtered_df.index
X_cnn_spec_filtered = X_cnn_spectrograms[idx_filter]
y_cnn_labels = data_cnn_filtered_df['target_emotion'].map({'anger': 1, 'fear': 1, 'calmness': 0}).values
groups_cnn_labels = data_cnn_filtered_df['participant'].values

if np.isnan(X_cnn_spec_filtered).any():
    print("Warning: NaNs found in X_cnn_spec_filtered. Replacing with 0.")
    X_cnn_spec_filtered = np.nan_to_num(X_cnn_spec_filtered, nan=0.0)




Extracting CNN-style spectrogram features...
Shape of combined X_cnn_spectrograms: (414, 33, 7, 16)


In [26]:
# --- Model Definitions ---

def create_simple_cnn_model(input_shape, spec_augment_params=None):
    inputs = Input(shape=input_shape)
    x = inputs
    if spec_augment_params:
        x = SpecAugment(**spec_augment_params)(x)
    
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x); x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x) # (None, 16, 3, 32) for (33,7) input
    x = Dropout(0.3)(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x); x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x) # (None, 8, 1, 64)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    # Compile outside or pass optimizer, loss
    return model

def resnet_block_mod(input_tensor, hp, stage_idx, block_idx): # Modified for KerasTuner
    filters = hp.Int(f's{stage_idx}_b{block_idx}_filters', 32, 128, step=32)
    kernel_size = hp.Choice(f's{stage_idx}_b{block_idx}_kernel', [3, 5])
    strides = 1
    use_batchnorm = hp.Boolean(f's{stage_idx}_b{block_idx}_bn')

    x = Conv2D(filters, kernel_size=(kernel_size,kernel_size), strides=strides, padding='same')(input_tensor)
    if use_batchnorm: x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, kernel_size=(kernel_size,kernel_size), padding='same')(x)
    if use_batchnorm: x = BatchNormalization()(x)

    if strides > 1 or input_tensor.shape[-1] != filters: # Basic shortcut handling
        shortcut = Conv2D(filters, kernel_size=(1,1), strides=strides, padding='same')(input_tensor)
        if use_batchnorm: shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = input_tensor
    x = Add()([x, shortcut]); x = ReLU()(x)
    return x

def build_resnet_for_tuner(hp, input_shape, spec_augment_params=None): # For KerasTuner
    inputs = Input(shape=input_shape)
    x = inputs
    if spec_augment_params and hp.Boolean("spec_augment"):
         x = SpecAugment(freq_mask_param=hp.Int('SA_freq_param', 1, 10), # Example HP for SA
                         time_mask_param=hp.Int('SA_time_param', 1, 3),
                         num_freq_masks=hp.Int('SA_n_freq', 1, 2),
                         num_time_masks=hp.Int('SA_n_time', 1, 2))(x)


    initial_filters = hp.Int('initial_filters', 32, 64, step=32)
    x = Conv2D(initial_filters, (3,3), padding='same')(x)
    if hp.Boolean("initial_bn"): x = BatchNormalization()(x)
    x = ReLU()(x)

    num_stages = hp.Int('num_stages', 1, 2) # Simpler ResNet
    for s_idx in range(num_stages):
        num_blocks_in_stage = hp.Int(f's{s_idx}_num_blocks', 1, 2)
        for b_idx in range(num_blocks_in_stage):
            x = resnet_block_mod(x, hp, s_idx, b_idx)
        if s_idx < num_stages -1 and x.shape[1] > 1 and x.shape[2] > 1 : # Downsample if not last stage and possible
             if hp.Boolean(f"s{s_idx}_downsample"):
                x = MaxPooling2D((2,2))(x)


    x = Flatten()(x)
    x = Dense(units=hp.Int('dense_units', 32, 128, step=32), activation='relu')(x)
    x = Dropout(hp.Float('dropout_dense', 0.2, 0.5, step=0.1))(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_adapted_efficientnet(input_shape, num_classes=1, spec_augment_params=None):
    inputs = Input(shape=input_shape)  # e.g. (33,7,16)
    x = inputs

    # If channels ≠ 3, first collapse your 16 channels into 1, then convert to RGB
    if input_shape[-1] != EFFNET_CHANNELS:
        # 1) Mean across last axis → shape (H,W,1)
        x = Lambda(lambda t: tf.reduce_mean(t, axis=-1, keepdims=True),
                   name='collapse_channels')(x)
        # 2) Convert grayscale→RGB (now (H,W,3))
        x = Lambda(lambda t: tf.image.grayscale_to_rgb(t),
                   name='to_rgb')(x)

    # 3) Resize spatial dims to EfficientNet’s minimum
    x = Resizing(EFFNET_IMG_SIZE, EFFNET_IMG_SIZE, name='resize_eff')(
        x
    )

    # 4) (Optional) SpecAugment, after resizing but before feeding into EffNet
    if spec_augment_params:
        # clamp mask sizes so they’re meaningful on resized input
        sa_params = spec_augment_params.copy()
        sa_params['freq_mask_param'] = min(sa_params['freq_mask_param'], EFFNET_IMG_SIZE//4)
        sa_params['time_mask_param'] = min(sa_params['time_mask_param'], EFFNET_IMG_SIZE//4)
        x = SpecAugment(**sa_params)(x)

    # 5) Load pre‐trained EfficientNetB0, starting from this tensor `x`
    base = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_tensor=x,
        pooling='avg'
    )
    base.trainable = False

    # 6) Attach your own classification head
    preds = Dense(
        num_classes,
        activation='sigmoid' if num_classes==1 else 'softmax',
        name='effnet_preds'
    )(base.output)

    return Model(inputs=inputs, outputs=preds, name='AdaptedEfficientNetB0')

In [14]:
# --- EEGNet Model ---
# (Adapted from official EEGNet Keras implementation, simplified)
def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout')

    input1 = Input(shape=(Chans, Samples, 1)) # channels_first like format

    block1 = Conv2D(F1, (1, kernLength), padding='same', input_shape=(Chans, Samples, 1), use_bias=False)(input1)
    block1 = BatchNormalization(axis=1)(block1) # Normalizing over channels
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.))(block1)
    block1 = BatchNormalization(axis=1)(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropoutType(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization(axis=1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropoutType(dropoutRate)(block2)

    flatten = Flatten(name='flatten')(block2)
    dense = Dense(nb_classes, name='dense', kernel_constraint=max_norm(norm_rate))(flatten)
    softmax = Activation('sigmoid' if nb_classes == 1 else 'softmax', name='softmax')(dense) # Sigmoid for binary

    return Model(inputs=input1, outputs=softmax)


### KerasTuner Setup (Example for one fold)

In [15]:
# --- KerasTuner Example ---
# Best to run this separately or for a single fold due to time constraints.
# We'll define it but not run it in the main CV loop for all models to save time here.
def run_kerastuner_example(X_train_fold, y_train_fold, X_val_fold, y_val_fold, input_shape):
    print("\n--- Running KerasTuner Example ---")
    # Define SpecAugment parameters to be potentially tuned or fixed
    spec_augment_params_for_tuner = {
        'freq_mask_param': 5, # Max size of freq mask (fixed for this example)
        'time_mask_param': 2, # Max size of time mask (fixed for this example)
        'num_freq_masks': 1,
        'num_time_masks': 1
    }
    
    tuner = kt.Hyperband(
        lambda hp: build_resnet_for_tuner(hp, input_shape, spec_augment_params_for_tuner), # Pass fixed SA params
        objective='val_accuracy',
        max_epochs=10, # For demo, keep low. Real tuning: 30-50+
        factor=3,
        directory='kerastuner_dir',
        project_name='stress_resnet_tuning'
    )
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    print("Starting KerasTuner search...")
    tuner.search(X_train_fold, y_train_fold,
                 epochs=15, # Max epochs per trial for tuner search
                 validation_data=(X_val_fold, y_val_fold),
                 callbacks=[stop_early],
                 batch_size=16) # Tuner might override batch_size if defined in HP space

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"\nBest HPs found: {best_hps.values}")

    # Build the model with the best HPs
    # model = tuner.hypermodel.build(best_hps)
    # history = model.fit(X_train_fold, y_train_fold, epochs=20, validation_data=(X_val_fold, y_val_fold)) # Retrain
    # val_acc_per_epoch = history.history['val_accuracy']
    # best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
    # print(f'Best epoch: {best_epoch}')
    # Re-initialize the model and train for the optimal number of epochs on the full training data.
    # For now, just printing HPs.
    return best_hps

# To run the tuner (example using first fold data for X_cnn_spec_filtered):
# cv_temp = GroupKFold(n_splits=5)
# train_idx_kt, val_idx_kt = next(cv_temp.split(X_cnn_spec_filtered, y_cnn_labels, groups_cnn_labels))
# X_train_kt, X_val_kt = X_cnn_spec_filtered[train_idx_kt], X_cnn_spec_filtered[val_idx_kt]
# y_train_kt, y_val_kt = y_cnn_labels[train_idx_kt], y_cnn_labels[val_idx_kt]
# # best_hyperparameters = run_kerastuner_example(X_train_kt, y_train_kt, X_val_kt, y_val_kt, X_cnn_spec_filtered.shape[1:])
# # print("To use best HPs, manually update model creation or load them.")


In [27]:

# --- Training and Evaluation Loop for DL Models ---
def run_dl_model_cv(model_fn, X_data_list, y_data, groups_data, model_name,
                    n_splits=5, epochs=25, batch_size=16, compile_kwargs=None, model_kwargs=None):
    print(f"\n--- Evaluating: {model_name} ---")
    cv_dl = GroupKFold(n_splits=n_splits)
    scores, runtimes = [], []

    if model_kwargs is None:
        model_kwargs = {}
    X_for_split = X_data_list[0]

    for fold, (train_idx, test_idx) in enumerate(cv_dl.split(X_for_split, y_data, groups_data)):
        print(f"{model_name} - Fold {fold+1}/{n_splits}...")
        X_train = [x_mod[train_idx] for x_mod in X_data_list]
        X_test  = [x_mod[test_idx]  for x_mod in X_data_list]
        y_train, y_test = y_data[train_idx], y_data[test_idx]

        # clear any previous TF graph & free memory
        tf.keras.backend.clear_session()
        gc.collect()

        # instantiate a fresh model
        model = model_fn(**model_kwargs)

        # compile: either clone your optimizer or fall back to a new Adam each fold
        if compile_kwargs:
            # shallow-copy so we don't mutate the user's dict
            ck = compile_kwargs.copy()

            if 'optimizer' in ck:
                opt = ck.pop('optimizer')
                # re-create a fresh optimizer instance from its config
                ck['optimizer'] = type(opt).from_config(opt.get_config())

            model.compile(**ck)
        elif not model._is_compiled:
            model.compile(
                optimizer=Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'],
                run_eagerly=True
            )

        # fit + time it
        start_time = time.time()
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=0
        )
        runtime = time.time() - start_time

        # evaluate
        loss_val, accuracy = model.evaluate(X_test, y_test, verbose=0)
        scores.append(accuracy)
        runtimes.append(runtime)

        best_val_acc_fold = max(history.history.get('val_accuracy', [0]))
        print(f"Fold {fold+1}: Acc={accuracy:.4f}, Time={runtime:.2f}s. "
              f"Best val_acc: {best_val_acc_fold:.4f}")

    # summarize
    mean_acc = np.mean(scores) if scores else np.nan
    std_acc  = np.std(scores)  if scores else np.nan
    mean_rt  = np.mean(runtimes) if runtimes else np.nan

    print(f"{model_name} CV Results: Mean Acc = {mean_acc:.4f} "
          f"+/- {std_acc:.4f}, Mean Runtime = {mean_rt:.2f}s")

    return [model_name, mean_acc, std_acc, mean_rt]


In [28]:
# --- Run DL Models ---
dl_results = []
N_SPLITS_DL = 3     # e.g. 3 folds
EPOCHS_DL   = 15    # e.g. 15 epochs for faster testing
BATCH_SIZE_DL = 16


def create_fixed_resnet_like_model(input_shape, spec_augment_params=None, num_blocks_list=[1,1], initial_filters=32):
    inputs = Input(shape=input_shape)
    x = inputs
    if spec_augment_params: x = SpecAugment(**spec_augment_params)(x)
    
    x = Conv2D(initial_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x); x = ReLU()(x)

    current_filters = initial_filters
    for i, num_blocks in enumerate(num_blocks_list):
        for _ in range(num_blocks):
            # Simplified ResNet block (no HP tuning here)
            y = Conv2D(current_filters, (3,3), padding='same')(x)
            y = BatchNormalization()(y); y = ReLU()(y)
            y = Conv2D(current_filters, (3,3), padding='same')(y)
            y = BatchNormalization()(y)
            if x.shape[-1] != current_filters: # Adjust shortcut
                x_shortcut = Conv2D(current_filters, (1,1), padding='same')(x)
            else:
                x_shortcut = x
            x = Add()([x_shortcut, y]); x = ReLU()(x)
        if i < len(num_blocks_list) - 1 and x.shape[1] > 1 and x.shape[2] > 1:
            current_filters *= 2 # Increase filters for next stage
            # Downsample for next stage (example, could be strided conv in block)
            x_ident = x # store for potential shortcut across pooling if needed
            x = MaxPooling2D((2,2))(x)
            # if a shortcut needs to cross pooling, it must also be pooled/projected.

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x) # Reduced dense units
    x = Dropout(0.4)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    # model.compile(...) done by run_dl_model_cv if needed
    return model

spec_augment_config = {
    'freq_mask_param': 5,
    'time_mask_param': 2,
    'num_freq_masks': 1,
    'num_time_masks': 1
}

# shared compile settings
compile_kwargs = {
    'optimizer': Adam(learning_rate=1e-3),
    'loss': 'binary_crossentropy',
    'metrics': ['accuracy'],
    'run_eagerly': True
}

# 1. Simple CNN + SpecAugment
dl_results.append(
    run_dl_model_cv(
        model_fn=create_simple_cnn_model,
        X_data_list=[X_cnn_spec_filtered],
        y_data=y_cnn_labels,
        groups_data=groups_cnn_labels,
        model_name="SimpleCNN_SpecAug",
        n_splits=N_SPLITS_DL,
        epochs=EPOCHS_DL,
        batch_size=BATCH_SIZE_DL,
        compile_kwargs=compile_kwargs,
        model_kwargs={
            'input_shape': X_cnn_spec_filtered.shape[1:],
            'spec_augment_params': spec_augment_config
        }
    )
)

# 2. ResNet-like + SpecAugment
dl_results.append(
    run_dl_model_cv(
        model_fn=create_fixed_resnet_like_model,
        X_data_list=[X_cnn_spec_filtered],
        y_data=y_cnn_labels,
        groups_data=groups_cnn_labels,
        model_name="ResNetLike_SpecAug",
        n_splits=N_SPLITS_DL,
        epochs=EPOCHS_DL,
        batch_size=BATCH_SIZE_DL,
        compile_kwargs=compile_kwargs,
        model_kwargs={
            'input_shape': X_cnn_spec_filtered.shape[1:],
            'spec_augment_params': spec_augment_config,
            'initial_filters': 32,
            'num_blocks_list': [1, 1]
        }
    )
)

# 3. Adapted EfficientNetB0 + SpecAugment (with a smaller LR)
dl_results.append(
    run_dl_model_cv(
        model_fn=create_adapted_efficientnet,
        X_data_list=[X_cnn_spec_filtered],
        y_data=y_cnn_labels,
        groups_data=groups_cnn_labels,
        model_name="AdaptedEfficientNetB0_SA",
        n_splits=N_SPLITS_DL,
        epochs=EPOCHS_DL,
        batch_size=BATCH_SIZE_DL,
        compile_kwargs={
            'optimizer': Adam(learning_rate=1e-4),
            'loss': 'binary_crossentropy',
            'metrics': ['accuracy'],
            'run_eagerly': True
        },
        model_kwargs={
            'input_shape': X_cnn_spec_filtered.shape[1:],
            'num_classes': 1,
            'spec_augment_params': spec_augment_config
        }
    )
)



--- Evaluating: SimpleCNN_SpecAug ---
SimpleCNN_SpecAug - Fold 1/3...


C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(16, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(10, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(sha

Fold 1: Acc=0.7083, Time=7.35s. Best val_acc: 0.7083
SimpleCNN_SpecAug - Fold 2/3...
Fold 2: Acc=0.6042, Time=7.09s. Best val_acc: 0.7083
SimpleCNN_SpecAug - Fold 3/3...
Fold 3: Acc=0.6905, Time=7.12s. Best val_acc: 0.7143
SimpleCNN_SpecAug CV Results: Mean Acc = 0.6677 +/- 0.0455, Mean Runtime = 7.19s

--- Evaluating: ResNetLike_SpecAug ---
ResNetLike_SpecAug - Fold 1/3...


C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(16, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(10, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(sha

Fold 1: Acc=0.6667, Time=12.80s. Best val_acc: 0.6875
ResNetLike_SpecAug - Fold 2/3...
Fold 2: Acc=0.6667, Time=12.80s. Best val_acc: 0.7083
ResNetLike_SpecAug - Fold 3/3...
Fold 3: Acc=0.7381, Time=13.29s. Best val_acc: 0.7381
ResNetLike_SpecAug CV Results: Mean Acc = 0.6905 +/- 0.0337, Mean Runtime = 12.96s

--- Evaluating: AdaptedEfficientNetB0_SA ---
AdaptedEfficientNetB0_SA - Fold 1/3...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(16, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(10, 33, 7, 16))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(sha

Fold 1: Acc=0.6667, Time=56.26s. Best val_acc: 0.6667
AdaptedEfficientNetB0_SA - Fold 2/3...
Fold 2: Acc=0.6667, Time=56.16s. Best val_acc: 0.6667
AdaptedEfficientNetB0_SA - Fold 3/3...
Fold 3: Acc=0.6667, Time=58.98s. Best val_acc: 0.6905
AdaptedEfficientNetB0_SA CV Results: Mean Acc = 0.6667 +/- 0.0000, Mean Runtime = 57.13s


In [30]:
# --- Data Prep and Evaluation for EEGNet ---
print("\nExtracting EEG data for EEGNet...")
# EEGNet typically uses (Batch, Channels, Samples, 1) or (Batch, 1, Channels, Samples)
# Our feat_extract_EEG with EEGNET_INPUT gives (Batch, Samples, Channels)
X_eegnet_raw = feat_extract_EEG(raw, model_type='EEGNET_INPUT', sampling_rate=EEGNET_SAMPLING_RATE, fixed_length=EEGNET_SAMPLES_PER_SUBJECT)

# Filter based on selected emotions (same indices as for CNNs)
X_eegnet_filtered = X_eegnet_raw[idx_filter] # idx_filter from CNN data prep

# Reshape for EEGNet: (Batch, Channels, Samples, 1) - Keras default for Conv2D is channels_last, but EEGNet paper implies specific convs
# The EEGNet implementation used here has axis=1 for BN, implying channels_first-like operations
# So we need (Batch, Chans, Samples, 1)
X_eegnet_reshaped = X_eegnet_filtered.reshape(X_eegnet_filtered.shape[0], EEGNET_CHANNELS, EEGNET_SAMPLES_PER_SUBJECT, 1)
# Transpose if feat_extract_EEG gave (Batch, Samples, Channels) -> (Batch, Channels, Samples) then expand_dims
# X_eegnet_reshaped = np.transpose(X_eegnet_filtered, (0, 2, 1))[:, :, :, np.newaxis]


print(f"Shape of X_eegnet_reshaped: {X_eegnet_reshaped.shape}") # Should be (num_filtered_samples, 14, 128, 1)
# y_eegnet_labels and groups_eegnet_labels are same as y_cnn_labels, groups_cnn_labels

if np.isnan(X_eegnet_reshaped).any():
    print("Warning: NaNs found in X_eegnet_reshaped. Replacing with 0.")
    X_eegnet_reshaped = np.nan_to_num(X_eegnet_reshaped, nan=0.0)


# 4. EEGNet
# EEGNet model expects specific input shape (Chans, Samples, 1) for the model definition provided
eegnet_input_shape = (EEGNET_CHANNELS, EEGNET_SAMPLES_PER_SUBJECT, 1)

dl_results.append(
    run_dl_model_cv(
        model_fn=EEGNet,
        X_data_list=[ X_eegnet_reshaped ],      # wrap in list!
        y_data=y_cnn_labels,
        groups_data=groups_cnn_labels,
        model_name="EEGNet",
        n_splits=N_SPLITS_DL,
        epochs=EPOCHS_DL + 10,                 # if you want more epochs
        batch_size=BATCH_SIZE_DL,
        compile_kwargs={
            'optimizer': Adam(learning_rate=1e-3),
            'loss': 'binary_crossentropy',
            'metrics': ['accuracy'],
            'run_eagerly': True
        },
        model_kwargs={
            'nb_classes': 1,
            'Chans': EEGNET_CHANNELS,
            'Samples': EEGNET_SAMPLES_PER_SUBJECT,
            'dropoutRate': 0.5
        }
    )
)



Extracting EEG data for EEGNet...
Shape of X_eegnet_reshaped: (138, 14, 128, 1)

--- Evaluating: EEGNet ---
EEGNet - Fold 1/3...


C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(16, 14, 128, 1))',)
  warnings.warn(msg)
C:\Users\ferri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `i

Fold 1: Acc=0.6667, Time=14.91s. Best val_acc: 0.6667
EEGNet - Fold 2/3...
Fold 2: Acc=0.6667, Time=14.77s. Best val_acc: 0.7292
EEGNet - Fold 3/3...
Fold 3: Acc=0.6667, Time=13.82s. Best val_acc: 0.6667
EEGNet CV Results: Mean Acc = 0.6667 +/- 0.0000, Mean Runtime = 14.50s


In [31]:
# --- Consolidate and Save Results ---
results_dl_df = pd.DataFrame(dl_results, columns=['name', 'mean_score', 'std_score', 'mean_runtime'])
print("\nDeep Learning Models CV Results:")
print(results_dl_df)
results_dl_df.to_csv('results_advanced_dl.csv', index=False)

print("\n--- Script Finished ---")


Deep Learning Models CV Results:
                       name  mean_score  std_score  mean_runtime
0         SimpleCNN_SpecAug    0.667659   0.045484      7.187518
1        ResNetLike_SpecAug    0.690476   0.033672     12.963567
2  AdaptedEfficientNetB0_SA    0.666667   0.000000     57.133949
3                    EEGNet    0.666667   0.000000     14.502123

--- Script Finished ---
